In [1]:
# Install necessary packages if missing 
!pip install arabic-reshaper python-bidi rich sacrebleu

In [2]:
import sys
sys.path.append("../src")

In [3]:
from llm.translation_pipeline import ArabicToGermanTranslator
from document_processing.processor import DocumentProcessor
import arabic_reshaper
from bidi.algorithm import get_display
from rich.console import Console
from sacrebleu import corpus_bleu
from bert_score import score as bert_score
import pandas as pd

In [4]:
# # Utility Functions
# def reshape_arabic(text: str) -> str:
#     reshaped_text = arabic_reshaper.reshape(text)
#     bidi_text = get_display(reshaped_text)
#     return bidi_text

### Test _Dummy_ Translation using BLEU and BERT Score

In [19]:
# Example Arabic Texts (could be extracted dynamically later)
arabic_texts = [
    "آية الدرس: \"تنضح علىَّ بزوفاك فأطهر. تغسلني فأَبيضَ أكثرَ من الثلج\" (مزمور 50: 7 قبطي)",
    "الأسبوع الثاني من شهر برمهات (مارس)",
    "الدرس السابع: القديسة دميانة",
    "قصة توبة",
    "الدرس القادم: ليلة أبوغلمسيس"
]

# Placeholder Human Translations (Later extracted from Word docs)
human_translations = [
    "Lehrvers: \"Entsündige mich mit Ysop, dann werde ich rein, wasche mich, dann werde ich weißer sein als Schnee.\" (Psalm 50:7 Koptisch)",
    "Die zweite Woche des Monats Baramhat (März)",
    "Siebte Lektion: Die heilige Damiana",
    "Geschichte der Reue",
    "Nächste Lektion: Die Nacht von Apocalypse (Abughalamsis-Nacht)"
]

# Translate and Compare
llm_translations = []

for arabic in arabic_texts:
    console.rule("Original Arabic")
    console.print(arabic, style="bold green")

    llm_output = translator.translate(arabic)
    llm_translations.append(llm_output)

    console.rule("LLM German Translation")
    console.print(llm_output, style="bold blue")

# --- Automated BLEU Evaluation ---
bleu = corpus_bleu(llm_translations, [human_translations])
print("\n--- BLEU Evaluation Results ---")
print(f"BLEU Score: {bleu.score:.2f}")

# --- BERTScore Evaluation ---
P, R, F1 = bert_score(llm_translations, human_translations, lang="de", verbose=True)

print("\n--- BERTScore Evaluation Results ---")
print(f"Average Precision: {P.mean():.4f}")
print(f"Average Recall:    {R.mean():.4f}")
print(f"Average F1:        {F1.mean():.4f}")

# Comparison Table
comparison_df = pd.DataFrame({
    "Arabic": arabic_texts,
    "Human German": human_translations,
    "LLM German": llm_translations,
    "BERTScore F1": F1.tolist()
})

comparison_df


───────────────────────────────────────────────── Original Arabic ─────────────────────────────────────────────────

آية الدرس: "تنضح علىَّ بزوفاك فأطهر. تغسلني فأَبيضَ أكثرَ من الثلج" (مزمور 50: 7 قبطي)

───────────────────────────────────────────── LLM German Translation ──────────────────────────────────────────────

Die Lektion lautet: "Du wirst mir dein Schwert geben, ich werde gereinigt, du wirst mich waschen, und ich werde 
weißer als Eis."

───────────────────────────────────────────────── Original Arabic ─────────────────────────────────────────────────

...

computing greedy matching.


C:\Users\kirol\miniconda3\envs\cagi_env\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.39 seconds, 12.69 sentences/sec

--- BERTScore Evaluation Results ---
Average Precision: 0.8323
Average Recall:    0.7751
Average F1:        0.8004


,Arabic,Human German,LLM German,BERTScore F1
0,"آية الدرس: ""تنضح علىَّ بزوفاك فأطهر. تغسلني فأ...","Lehrvers: ""Entsündige mich mit Ysop, dann werd...","Die Lektion lautet: ""Du wirst mir dein Schwert...",0.745416
1,الأسبوع الثاني من شهر برمهات (مارس),Die zweite Woche des Monats Baramhat (März),"Die zweite Woche des Monats, März.",0.851452
2,الدرس السابع: القديسة دميانة,Siebte Lektion: Die heilige Damiana,Lektion 7: Die Heilige Dame.,0.796391
3,قصة توبة,Geschichte der Reue,Eine Geschichte der Reue.,0.856793
4,الدرس القادم: ليلة أبوغلمسيس,Nächste Lektion: Die Nacht von Apocalypse (Abu...,Nächste Lektion:,0.751939


### Upload Sundyschool Lesson Translation

In [25]:
# Instantiate LLM Translator
translator = ArabicToGermanTranslator()
console = Console()

doc_path = "../docs/processed/Zweite Woche des Monats Baramhat (März)_Maria die Ägypterin.docx"
german_start = "===GERMAN START==="  # adjust based on your document

processor = DocumentProcessor(doc_path, german_start)
arabic_paragraphs, human_translations = processor.extract_text()

console.rule("Original Arabic")
for a in arabic_paragraphs:
    console.print(a, style="bold green")

console.rule("Human Translation")
for h in human_translations:
    console.print(h, style="bold blue")

Device set to use cuda:0


───────────────────────────────────────────────── Original Arabic ─────────────────────────────────────────────────

الأسبوع الثاني من شهر برمهات (مارس)
 الدرس السابع: القديسة دميانة
 قصة توبة
 الدرس القادم: ليلة أبوغلمسيس

مريم المصرية

المحور التعليمي: رحلة يوم (آبائي)

...

Sie verließ Jerusalem, überquerte den Jordan und lebte 47 Jahre in der Wüste in Gebet und Reue. Ihr Essen bestand 
aus Kräutern, und sie kämpfte stark gegen Versuchungen, doch Gott gab ihr Kraft.

Am Ende ihres Lebens begegnete ihr der heilige Zosimas, der sie auf wundersame Weise fand. Sie kannte seinen Namen 
ohne Vorstellung – durch den Heiligen Geist. Sie bat um die Kommunion und beichtete ihm ihr Leben.

Ein Jahr später kam er zurück und fand sie entschlafen, mit einer Nachricht neben ihrem Körper:
 „Begrabe Maria, die arme Sünderin, im Staub, aus dem sie gemacht ist. Und der Staub soll sich nicht rühmen.“

Sie lebte 76 Jahre. Die Kirche gedenkt ihrer am 1. Baramouda (9. April).

.

### Translation Lesson using LLM Model

In [21]:
# Translate and Compare
llm_translations = []

for arabic in arabic_paragraphs:  # limit for now
    llm_translation = translator.translate(arabic)
    llm_translations.append(llm_translation)

console.rule("LLM Translation")
for l in llm_translations:
    console.print(l, style="bold yellow")

───────────────────────────────────────────────── LLM Translation ─────────────────────────────────────────────────

Die zweite Woche im März, Lektion sieben: "Die Heilige Dame" die nächste Lektion:

Ägyptische Maria

Bildungszentrum: Die Reise zum Tag meiner Eltern

"Gott hat mich geliebt, hat mir die Tür der Reue geöffnet, niemand kann sie vor mir schließen, mit Freude am 
Kreuz."

...

Das ist das erste Mal, dass du mir ein Geschenk machst.

Doch als sie zwölf Jahre alt war, begann der Teufel, sie zu täuschen und zu verführen, und sie zog es vor, 17 Jahre
lang von unserem Herrn getrennt zu leben, und ihre Sünden waren sehr schwer, doch unser Herr wollte nicht, dass der
Sünder stirbt und die Liebe seiner Kinder weggerissen wird. Eines Sommers sah sie ein Schiff, das nach Jerusalem 
reiste, um dort das Kreuzfest zu besuchen, und da ging sie mit ihm hin, und als sie in der Auferstehungskirche war,
spürte sie, dass ihre Männer auf der Erde waren, und dass sie von der Kirche ferngehalten wurde, und dass sie von 
der Kirche ferngehalten wurde.

Magh Tani ging hinaus zu ihren Bewohnern oder zu ihren Arazien und in der Mitte von dem, was sie vor den Toren der 
Auferstehung sah, hörte sie eine Stimme zu ihr sagen: „Gehen Sie durch Jordanien und dort wird es Ihnen bequem.“ 
Sie ging also aus Jerusalem hinaus, überquerte den Jordan, nahm drei Silberklößchen mit, lebte drei Brote neben dem
Jordan in der Wildnis, lebte 47 Jahre in der Wüste und lebte 47 Jahre lang in der Wüste in einem Gebet für unseren 
Herrn, und er aß Gras, und die Heilige Maria befand sich in sehr schweren Kriegen des Teufels, und sie wurde 
besiegt.

Und am Ende ihres Lebens in der Wildnis traf sie einen Heiligen namens St. Zusema, der sie von Weitem heilte, und 
fragte ihn, ob er herausfinden würde, wer der Mensch sei, den sie in seinem Namen nannte, und er fragte ihn 
natürlich, wie sie seinen Namen kannte, aber er wartete darauf, dass die Heilige ihn mit ihren Tränen beweinte, um 
sie zu beruhigen, und sie fragte ihn, ob er sie in der Tat in der Erde begraben würde. Und tatsächlich kam er zu 
ihr und traf sie fast tot.

Die Kirche feiert ihren 1. Brief mit der Post, die immer dem 9. April zustimmt.

### Test LLM translation vs. Human translation using BLEU and BERT Score

In [24]:
# --- Automated BLEU Evaluation ---
bleu = corpus_bleu(llm_translations, [human_translations])
print("\n--- BLEU Evaluation Results ---")
print(f"BLEU Score: {bleu.score:.2f}")

print(f"LLM translations: {len(llm_translations)}")
print(f"Human translations: {len(human_translations)}")


# --- BERTScore Evaluation ---
P, R, F1 = bert_score(llm_translations, human_translations, lang="de", verbose=True)

print("\n--- BERTScore Evaluation Results ---")
print(f"Average Precision: {P.mean():.4f}")
print(f"Average Recall:    {R.mean():.4f}")
print(f"Average F1:        {F1.mean():.4f}")

# Comparison Table
comparison_df = pd.DataFrame({
    "Arabic": arabic_texts,
    "Human German": human_translations,
    "LLM German": llm_translations,
    "BERTScore F1": F1.tolist()
})

comparison_df


--- BLEU Evaluation Results ---
BLEU Score: 1.92
LLM translations: 29
Human translations: 34


AssertionError: Different number of candidates and references